In [1]:
#Dependancies
import matplotlib.pyplot as plt
import pandas as pd
import requests
from datetime import datetime
import seaborn as sns
sns.set(style="darkgrid")

In [2]:
# USE MARS INSIGHT PROJECT API TO CREATE DAILY WEATHER REPORT OF MARS 
# Documentation: https://api.nasa.gov/assets/insight/InSight%20Weather%20API%20Documentation.pdf

In [3]:
from api_key import nasa_key
nasa_key
base_url ='https://api.nasa.gov/insight_weather/?'
query_url = f'{base_url}api_key={nasa_key}&feedtype=json&ver=1.'

In [4]:
query_url

'https://api.nasa.gov/insight_weather/?api_key=x0vI6p6jUZL6wftZ2XccsfpVahCArjfeIgxdkqmR&feedtype=json&ver=1.'

In [5]:
nasa_response = requests.get(query_url)
nasa_json=nasa_response.json()


In [6]:
sol_keys=nasa_json["sol_keys"]
print(f'A sol is a Martian day, the data has up to date information on the last {len(sol_keys)} Sols.')

A sol is a Martian day, the data has up to date information on the last 7 Sols.


In [7]:
#since only most recent 7 sol's are avaiable need to make the [sol] dynamic
#dynamic dates
sol_key=nasa_json["sol_keys"]
sol_list=[]
for sol in sol_key:
    sol_list.append(sol)

In [13]:
# Convert Martian date to earth date for all sol articles
date_list=[]
numerical_dates=[]

for day in sol_list:
    utc_day=nasa_json[day]['First_UTC']
    utc=datetime.strptime(utc_day, '%Y-%m-%dT%H:%M:%SZ')
    date_formatted=utc.strftime("%d-%b-%Y")
    numeric_formatted = utc.strftime("%d-%m-%Y")
    date_list.append(date_formatted)
    numerical_dates.append(numeric_formatted)

In [14]:
numerical_dates

['13-04-2020',
 '14-04-2020',
 '15-04-2020',
 '16-04-2020',
 '17-04-2020',
 '18-04-2020',
 '19-04-2020']

In [10]:
# Metrics available for each Sol:
# Each sol has linear averages for Temperature (AT), horizontal wind speed (HWS) and atmostpheric pressure (PRE)
print(f'The Temperature of sol_1 is {nasa_json[sol_list[0]]["AT"]["av"]} F.')
print(f'The average horizontal windspeed of sol_1 is {nasa_json[sol_list[0]]["HWS"]["av"]} m/s.')
print(f'The average atmospheric pressure of sol_1 is {nasa_json[sol_list[0]]["PRE"]["av"]} Pa.')
# There is also a most common wind direction measure
print(f'wind direction: {nasa_json[sol_list[0]]["WD"]["most_common"]["compass_point"]}')

The Temperature of sol_1 is -59.095 F.
The average horizontal windspeed of sol_1 is 4.839 m/s.
The average atmospheric pressure of sol_1 is 660.348 Pa.
wind direction: WNW


In [11]:
# iterate through each sol
for i in range(0,7):
    print(f'{date_list[i]}: \n AT: {nasa_json[sol_list[i]]["AT"]["av"]} F \n HWS: {nasa_json[sol_list[i]]["HWS"]["av"]} m/s \n PRE: {nasa_json[sol_list[i]]["PRE"]["av"]} Pa.')

          

13-Apr-2020: 
 AT: -59.095 F 
 HWS: 4.839 m/s 
 PRE: 660.348 Pa.
14-Apr-2020: 
 AT: -59.356 F 
 HWS: 4.831 m/s 
 PRE: 658.527 Pa.
15-Apr-2020: 
 AT: -59.627 F 
 HWS: 4.762 m/s 
 PRE: 661.28 Pa.
16-Apr-2020: 
 AT: -59.388 F 
 HWS: 4.622 m/s 
 PRE: 662.632 Pa.
17-Apr-2020: 
 AT: -59.119 F 
 HWS: 4.637 m/s 
 PRE: 663.224 Pa.
18-Apr-2020: 
 AT: -61.677 F 
 HWS: 4.589 m/s 
 PRE: 664.757 Pa.


KeyError: 'PRE'

In [ ]:
# Save result into variables for data frame
sol_variables = [sol_list[i] for i in range(0,7)]
date_variables = [date_list[i] for i in range(0,7)]
temp_variables = [nasa_json[sol_list[i]]["AT"]["av"] for i in range(0,7)]
HWS_variables = [nasa_json[sol_list[i]]["HWS"]["av"] for i in range(0,7)]
PRE_variables = [nasa_json[sol_list[i]]["PRE"]["av"] for i in range(0,7)]

In [ ]:
mars_df=pd.DataFrame({'Sol':sol_variables,
                        'Date': date_variables,
                        'Average Temp':temp_variables,
                        'Horizontal Wind Speed':HWS_variables,
                         'Pressure':PRE_variables
                        })
mars_df

In [ ]:
# Export the mars data into a .csv.
mars_df.to_csv('Resources/mars.csv', encoding='utf-8',index=False)

In [ ]:
#use temperature to alter size of points
s= mars_df['Average Temp']


In [ ]:
fig = plt.figure()
ax = sns.scatterplot(x='Sol', y='Average Temp', size=s, sizes=(75, 200),data=mars_df)
ax.set_title('Avareage Temperature on Mars')
ax.set_xlabel('Sol (Martian Day)')
ax.set_ylabel('AT (F)')
fig.savefig('Visualizations/AT.png')

In [ ]:
fig = plt.figure()
ax = sns.scatterplot(x='Sol', y='Horizontal Wind Speed', size=s, sizes=(75, 200), legend=False,data=mars_df)
ax.set_title('Horizontal Wind Speed on Mars')
ax.set_xlabel('Sol (Martian Day)')
ax.set_ylabel('HWS (m/s)')
fig.savefig('Visualizations/HWS.png')

In [ ]:
fig = plt.figure()
ax = sns.scatterplot(x='Sol', y='Pressure', size=s, sizes=(75, 200), legend=False,data=mars_df)
ax.set_title('Atmospheric Pressure on Mars')
ax.set_xlabel('Sol (Martian Day)')
ax.set_ylabel('Pressure (Pa)')
fig.savefig('Visualizations/Pressure.png')